In [1]:
import tensorflow as tf
import polars as pl
import numpy as np

2023-11-01 08:57:47.531092: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pl.read_csv("./data/EEG_Eye_State_Classification.csv")
df.head()
df = df.filter(pl.col("eyeDetection")==1).extend(df.filter(pl.col("eyeDetection") == 0)[:6723]).sample(fraction=1.0, shuffle=True, seed=42)
df.head()


AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,eyeDetection
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
4291.79,3933.33,4243.59,4077.95,4332.82,4616.92,4094.87,4631.28,4226.67,4251.28,4252.82,4302.05,4677.95,4384.1,1
4293.33,3993.85,4280.51,4107.69,4332.82,4618.46,4058.97,4615.9,4200.51,4234.36,4184.1,4280.0,4599.49,4366.15,0
4290.26,3997.95,4264.62,4112.82,4340.51,4618.97,4070.77,4624.62,4191.28,4224.62,4207.18,4284.1,4595.9,4352.31,0
4265.64,3975.9,4263.59,4111.79,4340.0,4616.92,4063.08,4623.59,4217.95,4226.15,4183.59,4271.28,4594.87,4333.33,0
4309.74,4037.44,4267.69,4125.13,4352.82,4625.13,4078.97,4607.69,4208.21,4232.31,4210.26,4285.13,4626.67,4372.82,1


In [3]:
X = df.drop("eyeDetection").to_numpy()
y = df.get_column("eyeDetection").to_numpy()

X = tf.keras.utils.normalize(X).reshape(-1, 14, 1)

In [4]:

inputs = tf.keras.Input(shape=(14,1))

Dense1 = tf.keras.layers.Dense(64, activation = 'relu',kernel_regularizer=tf.keras.regularizers.l2())(inputs)

lstm_1=  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences = True))(Dense1)
drop = tf.keras.layers.Dropout(0.3)(lstm_1)
lstm_3=  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences = True))(drop)
drop2 = tf.keras.layers.Dropout(0.3)(lstm_3)
flat = tf.keras.layers.Flatten()(drop2)
Dense_2 = tf.keras.layers.Dense(128, activation = 'relu')(flat)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(Dense_2)

model = tf.keras.Model(inputs, outputs)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 14, 1)]           0         
                                                                 
 dense (Dense)               (None, 14, 64)            128       
                                                                 
 bidirectional (Bidirection  (None, 14, 512)           657408    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 14, 512)           0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 14, 256)           656384    
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 14, 256)           0     

In [5]:
def train_model(model,X, y, epoch = 2):

        opt_adam = tf.keras.optimizers.Adam(learning_rate=0.001)

        es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
        mc = tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
        lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 0.001 * np.exp(-epoch / 10.))
        
        model.compile(optimizer=opt_adam,
                  loss=['binary_crossentropy'],
                  metrics=['accuracy'])
        
        history = model.fit(X,y,
                        batch_size=20,
                        epochs=epoch,
                        validation_split=0.15,
                        callbacks=[es,mc,lr_schedule])
        
        return model,history

train_model(model, X, y, 2)

Epoch 1/2
119/572 [=====>........................] - ETA: 1:09 - loss: 0.7096 - accuracy: 0.4891

KeyboardInterrupt: 